In [ ]:
# TODO:
# - are all requirements really needed? Maybe have been from prior SIGMOD'20 paper requirements (pip freeze > requirements.txt)
# - making notebook trusted helped



import pandas as pd
import numpy as np
import scipy as sc
import re               # regular expression
from resample import bootstrap as res_bootstrap
from ast import literal_eval
from scipy.stats import bootstrap as scipybootstrap
from scipy.stats import gmean       # geometric mean

import altair as alt
# alt.renderers.enable('jupyterlab', embed_options={'renderer': 'svg'})             # uses SVG instead of PNG which creates problems on my MAC
# Avoids writing all the data to the notebook or disk. 
# Note that this may not work on some cloud-based Jupyter notebook services.
# alt.data_transformers.enable('data_server')                                        # prevents writing data into JSON file, and instead loads when needed for analysis. Done in order to save space. But does not work so easily

# Set Jupyter and Pandas to show 3 decimal places
# %precision 3
# TODO: what is the interpretation of above line?

pd.options.display.float_format = '{:,.3f}'.format
np.set_printoptions(precision=3)
# np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})            # TODO: does not work for lists..
def print(*args):
    __builtins__.print(*("%.3f" % a if isinstance(a, float) else a
                         for a in args))

In [ ]:
# Specify a consistent, useful theme for Altair
def theme_1(*args, **kwargs):
    labelFont = 'CMU Serif'
    labelFontSize = 12
    labelFontWeight = 'normal'
    markColor = '#82c6df'
    headerFont = labelFont
    headerFontWeight = 'bold'
    headerFontSize = 12
    titleFont = labelFont
    titleFontWeight = 'bold'
    titleFontSize = 14
    
    return {
        'config': {
            'background' : '#ffffff',
            
            'view': {
                'height': 450,
                'width': 700,
            },

            'title': {
                'anchor': 'center',
                'color': '#000000',
                'font': titleFont,
                'fontSize': titleFontSize,
                'fontWeight': titleFontWeight,
                'fontStyle': 'italic'
            },

            'arc': {'fill': markColor},
            'area': {'fill': markColor},
            'line': {
                'stroke': markColor, 
                'strokeWidth': 2
                    },
            'path': {'stroke': markColor},
            'rect': {'fill': markColor},
            'shape': {'stroke': markColor},
            'symbol': {
                'fill': markColor, 
                'size': 30
            },

            'axis': {
                'labelFont': labelFont,
                'labelFontSize': labelFontSize,
                'labelFontWeight': labelFontWeight,
                'titleFont': titleFont,
                'titleFontSize': titleFontSize,
                'titleFontWeight': titleFontWeight
            },

            'axisX': {
                #'labelAngle': 0,
                #'labelPadding': 4,
                'tickSize': 3,
                'titleFont': headerFont,
                'titleFontSize': headerFontSize,
                'titleFontWeight': headerFontWeight,
            },

            'axisY': {
                #'labelBaseline': 'middle',
                #'maxExtent': 45,
                #'minExtent': 45,
                'tickSize': 2,
                #'titleAlign': 'left',
                #'titleAngle': 0,
                #'titleX': -45,
                #'titleY': -11
                'titleFont': headerFont,
                'titleFontSize': headerFontSize,
                'titleFontWeight': headerFontWeight,
            },
            
            'header': {
                'labelFont': labelFont,
                'labelFontSize': labelFontSize,
                'labelFontWeight': labelFontWeight,
                'titleFont': headerFont,
                'titleFontSize': headerFontSize,
                'titleFontWeight': headerFontWeight,
            },           
            
            'legend': {
                'labelFont': labelFont,
                'labelFontSize': labelFontSize,
                #'symbolType': 'square',
                'titleFont': titleFont,
                'titleFontSize': titleFontSize,
                'titleFontWeight': titleFontWeight
            },

            'range': {
                # any color scheme from https://vega.github.io/vega/docs/schemes/#categorical 
                # in an object with scheme attribute. Alternatively an array of hex colors e.g. 
                # ['#ec8431', '#829eb1', '#c89d29', '#3580b1', '#adc839', '#ab7fb4']
                'category': {'scheme': 'tableau10'},
                'diverging': {'scheme': 'purpleorange'},
                'heatmap': {'scheme': 'blues'},
                'ordinal': {'scheme': 'blues'},
                'ramp': {'scheme': 'blues'},
            }
        }
    }
alt.themes.register('theme_1', theme_1)
alt.themes.enable('theme_1');

## One of these themes would work too
# alt.themes.enable('latimes')
# alt.themes.enable('default')


# Styling function for pandas dataframes that highlights values less than 0.05.
# To be used with stat_sign_df.style.applymap()
def color_df(val):
    significant = True if (val < 0.05) else False
    if significant:
        return 'background-color: yellow; font-weight: bold'
    else:
        return ""
    

# Bootstrap method, adapted from: http://www.jtrive.com/the-empirical-bootstrap-for-confidence-intervals-in-python.html
# TODO: remove, better to use the existing scipy method noew imported that also allows 'BCa' instead of 'percentage' (below is percentage feshly implemented)
# 'from scipy.stats import bootstrap as scipybootstrap'
def bootstrap(data, n=3000, func=np.mean):
    """
    Generate `n` bootstrap samples, evaluating `func`
    at each resampling. `bootstrap` returns a function,
    which can be called to obtain confidence intervals
    of interest.
    """
    simulations = list()
    sample_size = len(data)
    xbar_init = np.mean(data)
    for c in range(n):
        itersample = np.random.choice(data, size=sample_size, replace=True)
        simulations.append(func(itersample))
    simulations.sort()
    def ci(p):
        """
        Return 2-sided symmetric confidence interval specified
        by p. p is a percentage (i.e. 95%, 90% etc.)
        """
        u_pval = (1+p)/2.
        l_pval = (1-u_pval)
        l_indx = int(np.floor(n*l_pval))
        u_indx = int(np.floor(n*u_pval))
        return[simulations[l_indx],simulations[u_indx]]
    return(ci)

## Global Variables Setup

In [ ]:
# A set of constant global variables used throughout the notebook
num_questions = 32
modes = ['SQL', 'RD']
mode_to_name = {0: 'SQL', 1: 'RD'}

BOOTSTRAPCONFIDENCE = 0.95      # confidence level used for bootstrap
BOOTSTRAPMETHOD = 'BCa'         # method used for bootstrap, appears to be better than the textbook version also available as 'percentage'
BOOTSTRAPSAMPLES = 10000        # number of resamples

## Loading the data

In [ ]:
# This should point to our full study data when it is available
filename = 'data/users-table-pilot.csv'
df = pd.read_csv(filename)
df['pattern_order']= df['pattern_order'].apply(literal_eval)    # turn string to array
df

# TODO: what is "current_page"?

## Reformatting the dataframe to make later analysis easier

In [ ]:
# The following sequence reformats the data frame to have one question per row. That simplifies the later analysis.

# reshape df (melt, pivot) to bring multiple question times (e.g. 'q7_time') per row into separate rows
# https://towardsdatascience.com/wide-to-long-data-how-and-when-to-use-pandas-melt-stack-and-wide-to-long-7c1e0f462a98
df2 = df.melt(id_vars=['worker_id', 'sequence_num', 'pattern_order',
                       'q1', 'q2','q3', 'q4','q5', 'q6','q7', 'q8', 'q9', 'q10',
                       'q11', 'q12','q13', 'q14','q15', 'q16','q17', 'q18', 'q19', 'q20',
                       'q21', 'q22','q23', 'q24','q25', 'q26','q27', 'q28', 'q29', 'q30',
                       'q31', 'q32'], value_vars=['q1_time', 'q2_time', 'q3_time', 'q4_time','q5_time', 'q6_time', 'q7_time', 'q8_time', 'q9_time', 'q10_time',
                                                  'q11_time', 'q12_time', 'q13_time', 'q14_time', 'q15_time', 'q16_time', 'q17_time', 'q18_time', 'q19_time', 'q20_time',
                                                  'q21_time', 'q22_time', 'q23_time', 'q24_time', 'q25_time', 'q26_time', 'q27_time', 'q28_time', 'q29_time', 'q30_time',
                                                  'q31_time', 'q32_time'], var_name='question', value_name='time')

# replace time in msec with sec in column 'time'
df2['time'] = df2['time'] / 1000

# replace question string 'q7_time' with number '7' in column 'question'
new_column = []
for values in df2['question']:
    new_column.append(int(re.search(r'\d+', values).group()))
df2['question'] = new_column

# choose the right pattern from the list 'pattern_order' and add ass column 'pattern
new_column = []
for (pattern_order_list, ind) in zip(df2['pattern_order'], df2['question']):
    new_column.append(pattern_order_list[ind-1])
df2['pattern'] = new_column

# determine the 'mode' (SQL or RD) from 'sequence_num' and 'question'
#   sequence_num = 0 means that the first question is shown in SQL, 1 means we start instead with RD. Then alternate between the two.
#   Thus (sequence_num + question_num) % 2 == 1 means SQL
#   Thus (sequence_num + question_num) % 2 == 0 means RD
new_column = []
for (sequence, question) in zip(df2['sequence_num'], df2['question']):
    mode = 'SQL' if (sequence + question) % 2 == 1 else 'RD'
    new_column.append(mode)
df2['mode'] = new_column

# determine the 'choice' (among the 4 patterns) made by the user for this question. Requires all the 32 question choices (e.g. 'q7') and index of the question at hand ('question')
questionarray = df2[['q1', 'q2','q3', 'q4','q5', 'q6','q7', 'q8', 'q9', 'q10',
                     'q11', 'q12','q13', 'q14','q15', 'q16','q17', 'q18', 'q19', 'q20',
                     'q21', 'q22','q23', 'q24','q25', 'q26','q27', 'q28', 'q29', 'q30',
                     'q31', 'q32']].to_numpy()
questionindex = df2[["question"]].to_numpy()

new_array = np.take_along_axis(questionarray,questionindex-1,1)     # take the 'questionindex'-th entry from each row of teh questionarray (notice 1-index vs 0-indexin)
df2['choice'] = new_array

# determine whether the choice was correct by comparing the ground truth ('pattern') against the choice made ('choice'). Saved as 0/1 value in new column 'correct'
new_column = []
for (pattern, choice) in zip(df2['pattern'], df2['choice']):
    correct = 1 if pattern == choice else 0
    new_column.append(correct)
df2['correct'] = new_column

# sort by worker and question number, and reset the inde
df2.sort_values(by=['worker_id', 'question'], inplace=True)
df2.reset_index(drop=True, inplace=True)
# df2

# select only the relevant subset of columns
df3 = df2[['worker_id', 'question', 'time', 'pattern', 'mode', 'choice', 'correct']]
df3

In [ ]:
# df8 = df3.loc[df3['mode'] == 'SQL', df3['pattern'] == 1, ['time']]
df8 = df3.query("mode=='SQL' and pattern==2")['time']
# df.query('Fee==25000')['Courses']
display(df8)

## Outline of the questions asked / analysis performed later

### TODO: double-check the set of questions below, plus where the 90% confidence interval is calculated

TIMING PER PARTICIPANT TOTAL
1. per participant, calculate the mean time in seconds spent on RD (32/2=16 per participant, irrespective of correctness), and SQL, and their difference median(RD)-median(SQL)         [TODO: is mean here instead of median correct?]
2. calculate the median of those differences across n participants (-4.81 sec, thus faster with RD)
3. calculate the fraction of users who whose difference is < 0 (thus who were faster with RD)


TODO: correct the numbering scheme with consistent indentation. I like to use continuous numbering


TIMING PER PARTICIPANT 1st / 2nd half
4. per participant, calculate the mean time over all questions answered in 1st half in RD (32/2/2=8), and SQL and in 2nd half.
5. Calculate the relative improvement rate (in percentage) (1st-2nd)/1st for RD, and SQL, and their difference.

TIMING ACROSS PARTICIPANTS
6. take median over all questions and all users answered in SQL (32/2*13), or in QV (2 values)
7. Calculate p-value for significance of difference

TIMING PATTERNS ACROSS PARTICIPANTS
8. Report the median time per pattern (4) across the two modes (2). Thus 8 values. Also report differences

In [ ]:
# Answers to questions 1-3

# for each worker, calculate mean and median for both modes
dfq1 = df3.groupby(['worker_id', 'mode']).time.agg(['mean', 'median'])

# pivot to have one row per worker
dfq1 = pd.pivot_table(dfq1, values=['mean', 'median'], index=['worker_id'], columns=['mode'])

# add the differences between means and medians cross the modes
dfq1['diff mean'] = dfq1['mean','RD'] - dfq1['mean','SQL']
dfq1['diff median'] = dfq1['median','RD'] - dfq1['median','SQL']
dfq1['ratio mean'] = dfq1['mean','RD'] / dfq1['mean','SQL']
dfq1['ratio median'] = dfq1['mean','RD'] / dfq1['median','SQL']
display(dfq1)

In [ ]:
# Answers to questions 1-2

data = dfq1['diff mean']
print('mean of mean differences = ', np.mean(data))
# ci = scipybootstrap((data,), statistic=np.mean, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method=BOOTSTRAPMETHOD, axis=0).confidence_interval        #convert array to sequence
# print('95% CI = ', np.array([ci.low, ci.high]))
ci = scipybootstrap((data,), statistic=np.mean, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='percentile', axis=0).confidence_interval        #convert array to sequence
print('95% CI (percentile) = ', np.array([ci.low, ci.high]))
ci = scipybootstrap((data,), statistic=np.mean, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='BCa', axis=0).confidence_interval        #convert array to sequence
print('95% CI (BCa) = ', np.array([ci.low, ci.high]))
print()

data = dfq1['diff mean']
print('median of mean differences = ', np.median(data))
# ci = scipybootstrap((data,), statistic=np.median, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method=BOOTSTRAPMETHOD, axis=0).confidence_interval        #convert array to sequence
# print('95% CI = ', np.array([ci.low, ci.high]))
ci = scipybootstrap((data,), statistic=np.median, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='percentile', axis=0).confidence_interval        #convert array to sequence
print('95% CI (percentile) = ', np.array([ci.low, ci.high]))
ci = scipybootstrap((data,), statistic=np.median, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='BCa', axis=0).confidence_interval        #convert array to sequence
print('95% CI (BCa) = ', np.array([ci.low, ci.high]))
print("   Notice that 'BCa' is identical to 'percentile' for median!! ")
print()

data = dfq1['diff median']
print('median of median differences = ', np.median(data))
# ci = scipybootstrap((data,), statistic=np.median, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method=BOOTSTRAPMETHOD, axis=0).confidence_interval        #convert array to sequence
# print('95% CI = ', np.array([ci.low, ci.high]))
ci = scipybootstrap((data,), statistic=np.median, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='percentile', axis=0).confidence_interval        #convert array to sequence
print('95% CI (percentile) = ', np.array([ci.low, ci.high]))
ci = scipybootstrap((data,), statistic=np.median, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='BCa', axis=0).confidence_interval        #convert array to sequence
print('95% CI (BCa) = ', np.array([ci.low, ci.high]))
print()

data = dfq1['diff median']
print('mean of median differences = ', np.mean(data))
ci = scipybootstrap((data,), statistic=np.mean, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='percentile', axis=0).confidence_interval        #convert array to sequence
print('95% CI (percentile) = ', np.array([ci.low, ci.high]))
ci = scipybootstrap((data,), statistic=np.mean, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='BCa', axis=0).confidence_interval        #convert array to sequence
print('95% CI (BCa) = ', np.array([ci.low, ci.high]))
print()
print()


print('mean of mean RD = ', np.mean(dfq1['mean', 'RD']))
print('mean of mean SQL = ', np.mean(dfq1['mean', 'SQL']))
print('median of mean RD = ', np.median(dfq1['mean', 'RD']))
print('median of mean SQL = ', np.median(dfq1['mean', 'SQL']))
print('mean of median RD = ', np.mean(dfq1['median', 'RD']))
print('mean of median SQL = ', np.mean(dfq1['median', 'SQL']))
print('median of median RD = ', np.median(dfq1['median', 'RD']))
print('median of median SQL = ', np.median(dfq1['median', 'SQL']))

In [ ]:
# Relative improvements from RD over SQL

data = dfq1['ratio mean']
print('mean of ratio mean differences = ', np.mean(data))
ci = scipybootstrap((data,), statistic=np.mean, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='percentile', axis=0).confidence_interval        #convert array to sequence
print('95% CI (percentile) = ', np.array([ci.low, ci.high]))
ci = scipybootstrap((data,), statistic=np.mean, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='BCa', axis=0).confidence_interval        #convert array to sequence
print('95% CI (BCa) = ', np.array([ci.low, ci.high]))
print()

data = dfq1['ratio mean']
print('median of ratio mean differences = ', np.median(data))
ci = scipybootstrap((data,), statistic=np.median, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method=BOOTSTRAPMETHOD, axis=0).confidence_interval        #convert array to sequence
print('95% CI = ', np.array([ci.low, ci.high]))
print()

data = dfq1['ratio median']
print('median of ratio median differences = ', np.median(data))
ci = scipybootstrap((data,), statistic=np.median, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method=BOOTSTRAPMETHOD, axis=0).confidence_interval        #convert array to sequence
print('95% CI = ', np.array([ci.low, ci.high]))
print()

data = dfq1['ratio median']
print('mean of ratio median differences = ', np.mean(data))
ci = scipybootstrap((data,), statistic=np.mean, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='percentile', axis=0).confidence_interval        #convert array to sequence
print('95% CI (percentile) = ', np.array([ci.low, ci.high]))
ci = scipybootstrap((data,), statistic=np.mean, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='BCa', axis=0).confidence_interval        #convert array to sequence
print('95% CI (BCa) = ', np.array([ci.low, ci.high]))
print()

data = dfq1['ratio mean']
print('geometric mean of ratio mean differences = ', gmean(data))
ci = scipybootstrap((data,), statistic=np.mean, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='percentile', axis=0).confidence_interval        #convert array to sequence
print('95% CI (percentile) = ', np.array([ci.low, ci.high]))
ci = scipybootstrap((data,), statistic=np.mean, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='BCa', axis=0).confidence_interval        #convert array to sequence
print('95% CI (BCa) = ', np.array([ci.low, ci.high]))
print('as comparison: ratio of their means:', np.mean(dfq1['mean', 'RD']) / np.mean(dfq1['mean', 'SQL']))

print()

data = dfq1['ratio median']
print('geometric mean of ratio median differences = ', gmean(data))
ci = scipybootstrap((data,), statistic=np.mean, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='percentile', axis=0).confidence_interval        #convert array to sequence
print('95% CI (percentile) = ', np.array([ci.low, ci.high]))
ci = scipybootstrap((data,), statistic=np.mean, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method='BCa', axis=0).confidence_interval        #convert array to sequence
print('95% CI (BCa) = ', np.array([ci.low, ci.high]))
print()

In [ ]:
# Answers to question 3: Percentage being faster with RD

fastermean = np.sum(dfq1['diff mean']<0)/len(dfq1)
print('Percentage of users who were faster with RD via means = ', fastermean)
fastermedian = np.sum(dfq1['diff median']<0)/len(dfq1)
print('Percentage of users who were faster with RD via medians = ', fastermedian)

# Analysing improvements from 1st to 2nd half per mode

In [ ]:
# for each worker, calculate mean and median for both modes
dfq2 = df3.groupby(['worker_id', 'mode', df3['question']> 16]).time.agg(['median', 'mean'])         # TODO: it would be nicer to replace FALSE with 1st half and TRUE with 2nd half
    # dfq2.replace(False, '1st half')
    # dfq2['half'] = '2nd half' if dfq2['question'] == True else '1st half'

# pivot to have one row per worker
dfq2 = pd.pivot_table(dfq2, values=['mean', 'median'], index=['worker_id'], columns=['question', 'mode'])

# add the relative improvements between 1st half (FALSE) and 2nd half (TRUE) per mode
dfq2['improve mean RD'] = (dfq2['mean', True, 'RD'] - dfq2['mean', False, 'RD']) / dfq2['mean', False, 'RD']
dfq2['improve mean SQL'] = (dfq2['mean', True, 'SQL'] - dfq2['mean', False, 'SQL']) / dfq2['mean', False, 'SQL']
dfq2['improve median RD'] = (dfq2['median', True, 'RD'] - dfq2['median', False, 'RD']) / dfq2['median', False, 'RD']
dfq2['improve median SQL'] = (dfq2['median', True, 'SQL'] - dfq2['median', False, 'SQL']) / dfq2['median', False, 'SQL']
display(dfq2)

In [ ]:
# relative improvements from 1st to 2nd half
print('Relative improvements from 1st to 2nd half per mode')
print('[I now think this should be calculated differently, with some paired bootstrap]')

print("Suggested statistics (also from SIGMOD'20): median of mean differences:")
medianmeanRD = np.median(dfq2['improve mean RD'])
print('median of mean improvements RD = ', medianmeanRD)
medianmeanSQL = np.median(dfq2['improve mean SQL'])
print('median of mean improvements SQL = ', medianmeanSQL)
print('[I now think this should be calculated as follows]')
print('Ratio of medians for RD = ', (np.median(dfq2['mean', True, 'RD']) - np.median(dfq2['mean', False, 'RD'])) / np.median(dfq2['mean', False, 'RD']))
print('Ratio of medians for SQL = ', (np.median(dfq2['mean', True, 'SQL']) - np.median(dfq2['mean', False, 'SQL'])) / np.median(dfq2['mean', False, 'SQL']))
print()

print("Other non-suggested statistics:")
meanmeanRD = np.mean(dfq2['improve mean RD'])
print('mean of mean improvements RD = ', meanmeanRD)
meanmeanSQL = np.mean(dfq2['improve mean SQL'])
print('mean of mean improvements SQL = ', meanmeanSQL)
medianmedianRD = np.median(dfq2['improve median RD'])
print('median of median improvements RD = ', medianmedianRD)
medianmedianSQL = np.median(dfq2['improve median SQL'])
print('median of median improvements SQL = ', medianmedianSQL)

# TODO: what is here the right statistics and what is here the right confidence interval (perhaps on the differences?)

# Are relative improvements of RD over SQL constant across the two halfs

In [ ]:
# add the percentage improvements from SQL to RD per 1st and 2nd half
dfq2['perc mean 1st'] = (dfq2['mean', False, 'RD'] - dfq2['mean', False, 'SQL']) / dfq2['mean', False, 'SQL']
dfq2['perc median 1st'] = (dfq2['median', False, 'RD'] - dfq2['median',False, 'SQL']) / dfq2['median',False, 'SQL']
dfq2['perc mean 2nd'] = (dfq2['mean', True, 'RD'] - dfq2['mean', True, 'SQL']) / dfq2['mean', True, 'SQL']
dfq2['perc median 2nd'] = (dfq2['median', True, 'RD'] - dfq2['median',True, 'SQL']) / dfq2['median',True, 'SQL']

display(dfq2)

In [ ]:
# relative improvements of RD over SQL in 1st and 2nd half
print('Relative improvements of RD over SQL in 1st and 2nd half')

data = dfq2['perc mean 1st']
print('mean of percentage mean differences 1st = ', np.mean(data))
data = dfq2['perc mean 2nd']
print('mean of percentage mean differences 2nd = ', np.mean(data))
data = dfq2['perc median 1st']
print('mean of percentage median differences 1st = ', np.mean(data))
data = dfq2['perc median 2nd']
print('mean of percentage median differences 2nd = ', np.mean(data))
print()
data = dfq2['perc mean 1st']
print('median of percentage mean differences 1st = ', np.median(data))
data = dfq2['perc mean 2nd']
print('median of percentage mean differences 2nd = ', np.median(data))
data = dfq2['perc median 1st']
print('median of percentage median differences 1st = ', np.median(data))
data = dfq2['perc median 2nd']
print('median of percentage median differences 2nd = ', np.median(data))

# Statistics per mode across participants

In [ ]:
# for each worker, calculate mean and median for both modes
dfq3 = df3.groupby(['mode']).time.agg(['median', 'mean'])
display(dfq3)

print("Suggested statistics:")
for j in ('RD', 'SQL'):
    data = df3.query("mode=='{}'".format(j))['time']
    print('median of {} = '.format(j), np.median(data))
    ci = scipybootstrap((data,), statistic=np.median, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method=BOOTSTRAPMETHOD, axis=0).confidence_interval        #convert array to sequence
    print('95% CI = ', np.array([ci.low, ci.high]))
print()


print("Other statistics:")
for j in ('RD', 'SQL'):
    data = df3.query("mode=='{}'".format(j))['time']
    print('mean of {} = '.format(j), np.mean(data))
    ci = scipybootstrap((data,), statistic=np.mean, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method=BOOTSTRAPMETHOD, axis=0).confidence_interval        #convert array to sequence
    print('95% CI = ', np.array([ci.low, ci.high]))
print()


# Statistics per mode and pattern across participants

In [ ]:
# for each worker, calculate mean and median for both modes, then pivot the table
dfq4 = df3.groupby(['mode', 'pattern']).time.agg(['median', 'mean'])
dfq4 = pd.pivot_table(dfq4, values=['median', 'mean'], index=['pattern'], columns=['mode'])
display(dfq4)

print("Suggested statistics:")
for i in range(1, 5):
    for j in ('RD', 'SQL'):
        data = df3.query("mode=='{}' and pattern=={}".format(j, i))['time']
        print('pattern {} median of {} = '.format(i, j), np.median(data))
        ci = scipybootstrap((data,), statistic=np.median, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method=BOOTSTRAPMETHOD, axis=0).confidence_interval        #convert array to sequence
        print('95% CI = ', np.array([ci.low, ci.high]))
    print()

print("Non-recommended statistics:")
for i in range(1, 5):
    for j in ('RD', 'SQL'):
        data = df3.query("mode=='{}' and pattern=={}".format(j, i))['time']
        print('pattern {} mean of {} = '.format(i, j), np.mean(data))
        ci = scipybootstrap((data,), statistic=np.mean, n_resamples=BOOTSTRAPSAMPLES, confidence_level=BOOTSTRAPCONFIDENCE, method=BOOTSTRAPMETHOD, axis=0).confidence_interval        #convert array to sequence
        print('95% CI = ', np.array([ci.low, ci.high]))
    print()

# --- everything below is out for now ---



CORRECTNESS (to be done later)
1. We similarly computed the proportion of correct responses for each mode and their difference, as well as the proportion of correct responses for each mode on the first and second halves and the the difference of the two for each mode.

Then, across all participants, we calculated the median time difference and mean difference in proportion of correct responses. [TODO: explain]
??? We also computed the overall mean rate of improvement and mean difference in correct responses between the first and second halves. [TODO: details]

In [ ]:
def compute_per_user(user):
    incorrect_sql = []
    incorrect_rd = []  
    times_sql = []
    times_rd = []


    for i in range(0, num_questions):
        one_indexed_q_num = i + 1
        q_col = "q" + str(one_indexed_q_num)
        q_time_col = q_col + "_time"

        user_answer = user[q_col]
        user_time = user[q_time_col] / 1000 # ms to s
        answer = user['pattern_order'][i]

        incorrect_int = 1 if user_answer != answer else 0
        if (user['sequence_num'] + one_indexed_q_num) % 2 == 1:
            incorrect_sql.append(incorrect_int)
            times_sql.append(user_time)
        else:
            incorrect_rd.append(incorrect_int)
            times_rd.append(user_time)
            
    num_each_mode = int(num_questions / 2)
    halfway = int(num_each_mode / 2)

    # CORRECTNESS

    incorrect_sql_proportion = np.sum(incorrect_sql) / num_each_mode
    incorrect_rd_proportion = np.sum(incorrect_rd) / num_each_mode
    diff_incorrect_proportion = incorrect_rd_proportion - incorrect_sql_proportion
    
    first_incorrect_sql_proportion = np.sum(incorrect_sql[0:halfway]) / (num_each_mode / 2)
    first_incorrect_rd_proportion = np.sum(incorrect_rd[0:halfway]) / (num_each_mode / 2)
    first_diff_incorrect_proportion = first_incorrect_rd_proportion - first_incorrect_sql_proportion
                                                    
    second_incorrect_sql_proportion = np.sum(incorrect_sql[halfway:num_questions]) / (num_each_mode / 2)
    second_incorrect_rd_proportion = np.sum(incorrect_rd[halfway:num_questions]) / (num_each_mode / 2)
    second_diff_incorrect_proportion = second_incorrect_rd_proportion - second_incorrect_sql_proportion

    incorrectness_improvement_sql = first_incorrect_sql_proportion - second_incorrect_sql_proportion
    incorrectness_improvement_rd = first_incorrect_rd_proportion - second_incorrect_rd_proportion
    diff_incorrectness_improvement = incorrectness_improvement_rd - incorrectness_improvement_sql

    # TIMING

    median_sql_time = np.median(times_sql)                          # TODO: why median instead of mean per participant (mean is what we used in SIGMOD'20)
    median_rd_time = np.median(times_rd)
    diff_time = median_rd_time - median_sql_time


    first_median_sql_time = np.median(times_sql[0:halfway])
    first_median_rd_time = np.median(times_rd[0:halfway])
    first_diff_time = first_median_rd_time - first_median_sql_time
    
    second_median_sql_time = np.median(times_sql[halfway:num_questions])
    second_median_rd_time = np.median(times_rd[halfway:num_questions])
    second_diff_time = second_median_rd_time - second_median_sql_time
    
    time_improvement_sql = (first_median_sql_time - second_median_sql_time) / first_median_sql_time
    time_improvement_rd = (first_median_rd_time - second_median_rd_time) / first_median_rd_time
    diff_time_improvement = time_improvement_rd - time_improvement_sql


    return {
        'incorrect_sql_proportion' : incorrect_sql_proportion,
        'incorrect_rd_proportion' : incorrect_rd_proportion,
        'diff_incorrect_proportion' : diff_incorrect_proportion,        

        'first_incorrect_sql_proportion' : first_incorrect_sql_proportion,
        'first_incorrect_rd_proportion' : first_incorrect_rd_proportion,
        'first_diff_incorrect_proportion' : first_diff_incorrect_proportion,
        'second_incorrect_sql_proportion' : second_incorrect_sql_proportion,
        'second_incorrect_rd_proportion' : second_incorrect_rd_proportion,
        'second_diff_incorrect_proportion' : second_diff_incorrect_proportion,
        
        'median_sql_time' : median_sql_time,
        'median_rd_time' : median_rd_time,
        'diff_time' : diff_time,
        
        'first_median_sql_time' : first_median_sql_time,
        'first_median_rd_time' : first_median_rd_time,
        'first_diff_time' : first_diff_time,
        'second_median_sql_time' : second_median_sql_time,
        'second_median_rd_time' : second_median_rd_time,
        'second_diff_time' : second_diff_time,
        
        'time_improvement_sql' : time_improvement_sql,
        'time_improvement_rd' : time_improvement_rd,
        'diff_time_improvement' : diff_time_improvement,
        'incorrectness_improvement_sql' : incorrectness_improvement_sql,
        'incorrectness_improvement_rd' : incorrectness_improvement_rd,
        'diff_incorrectness_improvement' : diff_incorrectness_improvement
    }

In [ ]:
per_participant_df_stats = df.apply(lambda row: compute_per_user(row), axis='columns', result_type='expand')
per_participant_df = pd.concat([df, per_participant_df_stats], axis='columns')
per_participant_df


per_participant_df_stats


In [ ]:
np.median(per_participant_df['median_sql_time'])

In [ ]:
np.median(per_participant_df['median_rd_time'])

In [ ]:
np.median(per_participant_df['diff_time'])

# alt.Chart(per_participant_df['diff_time']).mark_point().encode(
#     x = 'median(diff_time:Q)'
# )

In [ ]:
np.median(per_participant_df['diff_incorrect_proportion'])

In [ ]:
alt_data = per_participant_df[['diff_time', 'diff_incorrect_proportion']]

In [ ]:
alt.Chart(alt_data).mark_tick().encode(
    x='diff_time:Q'
)

TODO: ALTAIR not workin

alt.Chart(alt_data).mark_tick().encode(
    x='diff_incorrect_proportion:Q'
)

In [ ]:
alt.Chart(alt_data).transform_density(
    'diff_time',
    as_=['diff_time', 'density'],
    extent=[np.min(alt_data['diff_time']), np.max(alt_data['diff_time'])]
).mark_area(orient='vertical').encode(
    x=alt.X(
        'diff_time:Q',
        title=None
    ),
    y=alt.Y(
        'density:Q',
        stack='center',
        impute=None,
        title=None,
        axis=alt.Axis(labels=False, values=[0],grid=False, ticks=True),
    )
).properties(
    width=400,
    height=100,
    title='Δ median time per question (Diagrams − SQL)'
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
)

In [ ]:
alt.Chart(alt_data).transform_density(
    'diff_incorrect_proportion',
    as_=['diff_incorrect_proportion', 'density'],
    extent=[np.min(alt_data['diff_incorrect_proportion']), np.max(alt_data['diff_incorrect_proportion'])]
).mark_area(orient='vertical').encode(
    x=alt.X(
        'diff_incorrect_proportion:Q',
        title=None,
    ),
    y=alt.Y(
        'density:Q',
        stack='center',
        impute=None,
        title=None,
        axis=alt.Axis(labels=False, values=[0],grid=False, ticks=True),
    )
).properties(
    width=400,
    height=60,
    title='Δ percent errors (Diagrams − SQL)'
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
)


In [ ]:
# Funtions used to calculate the confidence intervals and construct the graphs

# Returns as a dataframe the differrence of QV and Both modes from SQL together with their bca confidence intervals for a given metric.
def get_conf_interval_bca(values_per_mode, mode, column_list):
    
    values_per_with_bca_conf_intervals = {'value': [], 'mode': column_list, 'conf_interval_delta_from_value': []}
    
    if mode == 'median':
        function = np.median
    elif mode == 'gmean':
        function = sc.stats.gmean
    elif mode == 'mean':
        function = np.mean

    for mode in column_list:
        values_per_worker = values_per_mode[mode].values
        value = function(values_per_worker)
        values_per_with_bca_conf_intervals['value'].append(value)
        value_conf_interval = res_bootstrap.bootstrap_ci(values_per_worker, f=function, b=10000, ci_method="bca")

        # calculate the distance between the summary statistic and the lower and upper bounds of the confidence intervals
        left_delta = abs(value - value_conf_interval[0])
        right_delta = abs(value_conf_interval[1] - value)
        values_per_with_bca_conf_intervals['conf_interval_delta_from_value'].append((left_delta, right_delta))

    return pd.DataFrame(values_per_with_bca_conf_intervals)

# Returns an altair graph for the percentage differences + confidence interval graph
def get_conf_interval_graph(time_per_mode_with_bca_intervals, axis_domain, percent=False):
    axis_limits = alt.Scale(domain=axis_domain,zero=False)
        
    if percent:
        text_encoding = alt.Text('value:Q', format=".2%")
        x_encoding = alt.X('value:Q', title=None, scale=axis_limits, axis=alt.Axis(format='%'))
    else:
        text_encoding = alt.Text('value:Q', format=".2f")
        x_encoding = alt.X('value:Q', title=None, scale=axis_limits)
        
    points = alt.Chart().transform_calculate(
        percent_diff=alt.datum.percent_diff
    ).mark_point(
        filled=True,
        color='black'
    ).encode(
        x=x_encoding,
        color=alt.Color('mode:N', legend=None),
    )

    # generate the error bars
    errorbars = points.mark_rule(size=1).encode(
        x='xmin:Q',
        x2='xmax:Q',
        color=alt.Color('mode:N', legend=None)
    ).transform_calculate(
        xmin='datum.value-datum.conf_interval_delta_from_value[0]',
        xmax='datum.value+datum.conf_interval_delta_from_value[1]'
    )

    text = points.mark_text(
        align='center',
        baseline='middle',
        dy = -7,
        dx = 0,
        fontSize=8
    ).encode(
        text=text_encoding
    )

    graph = alt.layer(points, errorbars, text).facet(
        data=time_per_mode_with_bca_intervals,
        row=alt.Row(
            'mode:N',
            header=alt.Header(
    #             titleOrient='bottom',
    #             labelOrient='bottom',
    #             labelPadding=0,
                labelAngle=0,
                titlePadding=0,
                title=None
            ),
         sort=['SQL', 'QV', 'Both']
        ),
    ).configure_facet(
        spacing=0,
    ).configure_view(
        stroke=None,
        width=200,
    ).properties(
    #     title='Mean time per mode',
    ).configure_axis(
        gridOpacity=0.7
    #     orient = "top",
    )

    return graph

In [ ]:
median_time_per_mode_with_bca_conf_intervals = get_conf_interval_bca(mean_time_per_mode, 'median', modes)

In [ ]:
modes